<a href="https://colab.research.google.com/github/NamHofo/Project-02/blob/main/Web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import aiohttp
import aiofiles
import asyncio
import json
import re
from tqdm import tqdm
import pandas as pd
import random


In [54]:

# API endpoint
API_URL = "https://api.tiki.vn/product-detail/api/v1/products/{}"

# Số lượng sản phẩm mỗi file
BATCH_SIZE = 1000

# Chuẩn hóa nội dung description (loại bỏ HTML)
def clean_description(text):
    if not text:
        return ""
    text = re.sub(r'<[^>]+>', '', text)  # Xoá thẻ HTML
    text = re.sub(r'\s+', ' ', text).strip()  # Xóa khoảng trắng thừa
    return text

# Hàm lấy dữ liệu từ API
async def fetch_product(session, product_id):
    url = API_URL.format(product_id)
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                return {
                    "id": data.get("id"),
                    "name": data.get("name"),
                    "url_key": data.get("url_key"),
                    "price": data.get("price"),
                    "description": clean_description(data.get("description")),
                    "images_url": [img.get("base_url") for img in data.get("images", []) if "base_url" in img]
                }
            else:
                # Trả về lý do lỗi khi API không trả về status 200
                return {"id": product_id, "error": f"API returned status {response.status}"}
    except Exception as e:
        # Nếu có lỗi ngoài API (vd: timeout), ghi rõ lỗi
        return {"id": product_id, "error": f"Exception occurred: {str(e)}"}


# Hàm xử lý lấy dữ liệu theo batch
async def fetch_all_products(product_ids):
    tasks = []
    failed_products = []  # Danh sách các sản phẩm lỗi

    async with aiohttp.ClientSession() as session:
        for product_id in product_ids:
            tasks.append(fetch_product_with_delay(session, product_id))

        results = await asyncio.gather(*tasks)

    # Lọc sản phẩm hợp lệ và sản phẩm có lỗi
    successful_products = []
    for result in results:
        if isinstance(result, dict) and 'error' in result:  # Nếu có lỗi
            failed_products.append(result)  # Lưu sản phẩm có lỗi và lý do
        else:
            successful_products.append(result)  # Lưu sản phẩm hợp lệ

    return successful_products, failed_products


# Hàm với thời gian delay ngẫu nhiên
async def fetch_product_with_delay(session, product_id):
    # Thời gian delay ngẫu nhiên giữa 1 và 10 giây
    delay = random.randint(1, 5)
    print(f"Delaying for {delay} seconds before fetching product {product_id}")
    await asyncio.sleep(delay)  # Chờ trong thời gian ngẫu nhiên
    return await fetch_product(session, product_id)


# Hàm chia nhỏ danh sách thành từng batch
def split_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Hàm lưu dữ liệu vào file JSON
async def save_to_json(data, file_index):
    filename = f"products_{file_index}.json"
    async with aiofiles.open(filename, "w", encoding="utf-8") as f:
        await f.write(json.dumps(data, indent=4, ensure_ascii=False))
    print(f"Đã lưu: {filename}")

# Chạy toàn bộ quy trình
async def main():
    # 1. Đọc danh sách product_id từ file Excel
    df = pd.read_excel("products-0-200000.xlsx", engine="openpyxl")

    # 2. Lấy cột chứa product_id
    product_ids = df.iloc[:, 0].astype(str).tolist()

    # 3. Chỉ lấy 10 sản phẩm đầu tiên để test
    product_ids = product_ids[:10]

    print(f"Total products (for testing): {len(product_ids)}")

    # 4. Chia thành từng batch
    batches = list(split_list(product_ids, BATCH_SIZE))

    all_failed_products = []  # Danh sách lỗi tổng

    # 5. Tải dữ liệu theo batch
    for index, batch in enumerate(tqdm(batches, desc="Fetching data")):
        product_data, failed_products = await fetch_all_products(batch)

        # Lưu các sản phẩm hợp lệ
        await save_to_json(product_data, f"{index+1}")

        # Lưu sản phẩm lỗi
        all_failed_products.extend(failed_products)

    # 6. Lưu danh sách lỗi vào file riêng nếu có lỗi
    if all_failed_products:
        await save_to_json(all_failed_products, "errors")
        print(f"⚠️ Đã lưu danh sách lỗi vào errors.json")



In [55]:
# Chạy chương trình
import nest_asyncio
nest_asyncio.apply() # Apply nest_asyncio to allow nested event loops

if __name__ == "__main__":
    asyncio.run(main())

Total products (for testing): 10


Fetching data:   0%|          | 0/1 [00:00<?, ?it/s]

Delaying for 2 seconds before fetching product 1391347
Delaying for 4 seconds before fetching product 74897599
Delaying for 4 seconds before fetching product 154155413
Delaying for 4 seconds before fetching product 253117062
Delaying for 1 seconds before fetching product 130978358
Delaying for 4 seconds before fetching product 214009046
Delaying for 4 seconds before fetching product 171618108
Delaying for 2 seconds before fetching product 179970479
Delaying for 3 seconds before fetching product 139457837
Delaying for 4 seconds before fetching product 197334787


Fetching data: 100%|██████████| 1/1 [00:04<00:00,  4.35s/it]

Đã lưu: products_1.json
